In [3]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

https://arxiv.org/abs/1304.3061

10.1103/PhysRevX.6.031007

Second Quantized Hamiltonian
\begin{eqnarray*}
    \mathcal{H}(r)=h_0 + \sum_{pq} h_{pq}(r) a^{\dagger}_p a_q +\frac{1}{2} \sum_{pqrs} h_{pqrs}(r) a^{\dagger}_p a^{\dagger}_qa_ra_s
\end{eqnarray*}

\begin{eqnarray*}
    h_{pq}(r)=\int{d\mathbf{r}}\,\phi^*_p(\mathbf{r})\left(-\frac{1}{2}\nabla^2-\sum_{a}{\frac{Z_a}{\mathbf{r}_{a,\mathbf{r}}}}\right)\phi_q(\mathbf{r})
\end{eqnarray*}

\begin{eqnarray*}
    h_{pqrs}(r)=\int{d\mathbf{r_1}\,d\mathbf{r_2}}\,\phi^*_p(\mathbf{r_1})\phi^*_q(\mathbf{r_2})r_{1,2}^{-1}\phi_r(\mathbf{r_1})\phi_s(\mathbf{r_2})
\end{eqnarray*}

Jordan-Wigner transformation
\begin{eqnarray*}
    a^{\dagger} = I^{\otimes j-1}\otimes \sigma_{-} \otimes \sigma_{z}^{\otimes N-j}\\
    a = I^{\otimes j-1}\otimes \sigma_{+} \otimes \sigma_{z}^{\otimes N-j}
\end{eqnarray*}

In [4]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [5]:
II = ID.kron(ID)
XX = X.kron(X)
YY = Y.kron(Y)
ZZ = Z.kron(Z)
ZI = Z.kron(ID)
IZ = ID.kron(Z)

sig_is = np.kron([1, 1], [1, -1])
sig_si = np.kron([1, -1], [1, 1])

In [6]:
def repulsion_energy(Z1=1, Z2=1, r=75e-12):
    Eh = 4.3597447222071e-18 # hartree energy
    ep0 = 8.854187e-12
    e = -1.602176634e-19
    return (1/(4*pi*ep0)*(Z1*Z2*e**2)/r)/Eh

rep_energy = repulsion_energy()
print("Repulsion energy (Eh): %s"%rep_energy)

Repulsion energy (Eh): 0.7055696793059831


In [7]:
g0 = -0.4804
g1 = 0.3435
g2 = -0.4347
g3 = 0.5716
g4 = 0.0910
g5 = 0.0910

H = II*g0 + IZ*g1 + ZI*g2 + ZZ*g3 + XX*g4 + YY*g5

min(scipy.linalg.eig(H.get())[0])+rep_energy

(-1.145629444817661+0j)

In [8]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])

In [9]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum.fun+rep_energy)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxcc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting com

-1.145629444817662


/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:644: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[j] = func(sim[j])


In [10]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_is) # [zz, iz] = 0
    zi = expectation_value(result, sig_si) # [zz, zi] = 0
    return zz, iz, zi

def expectation_energy(params):
    xx =  sigma_xx(params)
    yy =  sigma_yy(params)
    zz, iz, zi =  sigma_zz(params)
    
    result = g0 + g1*iz + g2*zi + g3*zz + g4*xx + g5*yy
    return result

In [11]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum.fun+rep_energy)

-1.1456294448176618


In [12]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [13]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [14]:
lr = 0.1
err = 1
while err > 1e-5:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_10647/1088512639.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


3.2771275942132028
0.25087340996984175
0.21642798860086027
0.17299785149256727
0.127042900992303
0.08495812918109738
0.05122215078222822
0.027296428517020133
0.012076889658794665
0.003307506724051168
0.001234210214421605
0.0032382230817722957
0.0038293853294496527
0.0036995397718261325
0.0032503120313920886
0.0027019008242667453
0.0021662136382552544
0.0016931563188248289
0.0012988346634378312
0.000982184806577524
0.0007344376019901011
0.0005442638721736559
0.00040038835375455495
0.0002927686445351463
0.00021299570138316737
0.00015429889043244716
0.00011137199120508923
8.013731289757686e-05
5.750742577281809e-05
4.1171532305650516e-05
2.94159505212499e-05
2.0979269161391922e-05
1.4938675643283439e-05
1.0622518519409763e-05
7.5440569521911804e-06


In [15]:
energy = expectation_energy(params)
energy = energy + rep_energy
print(energy)

(-1.1456294446036503+0j)
